Install the tecton athena sdk

In [17]:
!pip install 'tecton[athena]==0.5.0b52'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.13.0
    Uninstalling protobuf-3.13.0:
      Successfully uninstalled protobuf-3.13.0
  Attempting uninstall: tecton
    Found existing installation: tecton 99.99.99
    Uninstalling tecton-99.99.99:
      Successfully uninstalled tecton-99.99.99
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.88.1 requires attrs==20.3.0, but you have attrs 19.2.0 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


Set Athena connector configurations

ATHENA_DATABASE - glue database that is used to store temporary tables that are queried by Tecton


In [8]:
import random
import string
import tecton
import tecton_athena

tecton.conf.set("ALPHA_ATHENA_COMPUTE_ENABLED", "true")
spine_table_name = "TEMP_SPINE_TABLE_FROM_DF_" + ''.join(random.choices(string.ascii_lowercase, k=5))
tecton.conf.set("ATHENA_SPINE_TEMP_TABLE_NAME", spine_table_name)
tecton.conf.set("ATHENA_DATABASE", "tecton_poc_db")

import tecton
tecton.version.summary()

Version: 0.5.0b28
Git Commit: 068e00ce483b51c1f5d23e445edb8f62e02314e0
Build Datetime: 2022-07-21T13:36:50


Vanguard has workgroups associated with Athena queries. In order to enforce the workgroup settings we specify the workgroup name.

In [2]:

session = tecton_athena.athena_session.get_session() 
config = session.config

config.workgroup = "camp-hung-aiml-Workgroup"

Everyone can create a workspace for themselves. In the sagemaker terminal go ahead and run the following commands:

1) tecton login --manual

When prompted for the cluster name you can specificy https://vanguard-poc.tecton.ai

You'll be prompted to copy and paste a url into the browser. That will provide you with a token that you then paste back into the terminal. If you haven't logged into tecton yet it'll ask you to logon

In order to authenticate with the Tecton service from the notebook you'll create an api key from the terminal

2) tecton api-key create

Copy and paste the api_key into the "TECTON_API_KEY" configuration below

In [3]:
tecton.conf.set("API_SERVICE" , "https://vanguard-poc.tecton.ai/api")
tecton.conf.set("TECTON_API_KEY", "fe981c75a30a9d716ecd1d896e5a3412")

You can then create your own workspace

2) tecton workspace create "your name" --live

The --live flag means that you've created a workspace that allows for materialization

You can now create a branch for yourself of the code commit repo and apply it to your workspace

3) tecton apply

tecton apply syncs what is in your branch of the repo with your development workspace

In [4]:
# you can replace prod here with the name of the workspace you created for your self
ws = tecton.get_workspace('prod')

# you can now see all of the feature views that have data
ws.list_feature_views()

['last_transaction_amount',
 'merchant_fraud_rate',
 'stock_price_feature_view',
 'transaction_user_is_adult',
 'user_credit_card_issuer',
 'user_date_of_birth',
 'user_distinct_merchant_transaction_count_30d',
 'user_home_location',
 'user_merchant_transaction_counts',
 'user_transaction_counts',
 'user_weekend_transaction_count_30d']

In [ ]:
ws = tecton.get_workspace("eddie")

fv = ws.get_feature_view("feature_view_global_age_avg")

ghf = fv.get_historical_features()

ghf.to_spark().show()

In [ ]:
data = [{"assets": "\"[100, 200, 300]\"", "persona_index": 1234}]

df = spark.createDataFrame(data)

df.show()

In [ ]:
fv = ws.get_feature_view("on_demand_feature_view_sum_of_assets")

result = fv.get_historical_features(spine=df)

result.to_spark().show()

In [ ]:
data = [{"gender": "F", "persona_index": 1234}]

df = spark.createDataFrame(data)

df.show()

In [ ]:
fv = ws.get_feature_view("on_demand_feature_view_one_hot_encoded_gender")
 
result = fv.get_historical_features(spine=df)
 
result.to_spark().show()

In [ ]:
import datetime;
  
# ct stores current time
ct = datetime.datetime.now()

data = [{"dob": "6-18-1983", "persona_index": 1234, "timestamp": ct}]

df = spark.createDataFrame(data)

df.show()

In [ ]:
fv = ws.get_feature_view("on_demand_feature_view_normalized_age")

result = fv.get_historical_features(spine=df)

result.to_spark().show()

In [ ]:
import datetime;
  
# ct stores current time
ct = datetime.datetime.now()

data = [{"persona_index": 1234, "assets": "\"[100, 200, 300]\"","gender": "M" ,"dob": "6-18-1983", "timestamp": ct } ]

df = spark.createDataFrame(data)

df.show()

In [ ]:
fs = ws.get_feature_service("ea_team_feature_service")

result = fs.get_historical_features(spine=df)

result.to_spark().show()